In [1]:
import os

In [2]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"

os.environ["FIREWORKS_API_KEY"] = "fw_3ZWqWSsKGFMvXM1qdAiTxXrw"

In [3]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(
    model="accounts/fireworks/models/llama-v3p2-3b-instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [4]:
from langchain_fireworks import FireworksEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = FireworksEmbeddings(
    model="nomic-ai/nomic-embed-text-v1.5",
)

In [8]:
from langchain import hub
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = PyMuPDFLoader(
    file_path="/Users/yichinw/Desktop/The-Almanack-of-Naval-Ravikant_Final.pdf"
)
docs = loader.load()

In [9]:
# RecursiveCharacterTextSplitter allows you to split based on sentence boundaries,
# and then split the sentences into chunks of a certain size, if the sentence is too long.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [10]:
vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

In [16]:
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence(
    [retrieve, generate]
)  # state graph lets you chain functions together and build your application
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [21]:
journal = input("how are you feeling today")

In [22]:
journal

"I'm feeling happy "

In [ ]:
response = graph.invoke({"question": journal})
print(response["answer"])

I don't know the answer to your question about what to do when feeling down. However, the text suggests that retirement is not just about having a lot of money, but also about finding something you love and being authentic. It's about creating wealth and freedom, but also about finding meaning and purpose in life.


In [28]:
with open("/Users/yichinw/Desktop/my_journal.txt", "w") as f:
    f.write(f"{journal} \n")
    f.write(response["answer"])